In [1]:
import trimesh
import numpy as np
import torch
import numpy as np
from scipy.spatial import cKDTree as KDTree
import pyrender
from PIL import Image
import os

In [21]:
def get_path(cat, inst, type='gt'):
    if type == 'gt':
        path = '/vision/vision_data/ShapeNetCore.v2/{}/{}/models/model_normalized.obj'.format(cat, inst)
    elif type == 'deepsdf':
        path = '/vision/hwjiang/3d_generation/DeepSDF/examples/all/Reconstructions_256/1500/Meshes/ShapeNetV2/{}/{}.ply'.format(cat, inst)
    elif type == 'deepls':
        path = './examples/all/Reconstructions/18/Meshes/ShapeNetV2/{}/{}.ply'.format(cat, inst)
    elif type == 'ours':
        path = './examples/all_computed_rot_quad_last/Reconstructions_fix/18/Meshes/ShapeNetV2/{}/{}.ply'.format(cat, inst)
    return path

def load_mesh(path):
    mesh = trimesh.load(path)
    # trimesh.repair.fix_inversion(mesh)
    # rot_angle = np.radians(-90)
    # rot_mat = trimesh.transformations.rotation_matrix(rot_angle, [1,0,0])
    # mesh.apply_transform(rot_mat)
    return mesh

def load_gt_mesh(path):
    load = fast_load_obj(open(path))
    mesh = trimesh.Trimesh(vertices=load[0]['vertices'], faces=load[0]['faces'])
    return mesh

In [3]:
def compute_normalization_parameters(mesh, buffer=1.03):
    if not isinstance(mesh, trimesh.Trimesh):
        raise ValueError("Input must be a trimesh.Trimesh object.")

    # Only consider vertices used in the faces of the mesh
    used_vertices = mesh.vertices[mesh.faces.flatten()]
    # Compute min and max in each dimension
    min_coords = used_vertices.min(axis=0)
    max_coords = used_vertices.max(axis=0)
    # Compute the center
    center = (max_coords + min_coords) / 2.0
    # Make the mean zero
    distances = np.linalg.norm(used_vertices - center, axis=1)
    max_distance = distances.max()
    # Add some buffer
    max_distance *= buffer
    return -center, 1.0 / max_distance


def normalize_mesh(mesh, scale, offset):
    if not isinstance(mesh, trimesh.Trimesh):
        raise ValueError("Input must be a trimesh.Trimesh object.")
    # Create a copy of the mesh to avoid modifying the original mesh
    normalized_mesh = mesh.copy()
    # Normalize the vertices
    normalized_mesh.vertices = (normalized_mesh.vertices / scale) - offset
    return normalized_mesh

In [16]:
mesh_dict = mesh_gt.dump()

(20,)

In [30]:
mesh_gt.vertices.min(), mesh_deepsdf_norm.vertices.min(), mesh_deepls_norm.vertices.min(), mesh_recon_norm.vertices.min(), 

(-0.393719, -0.38468157184087437, -0.39943887559223373, -0.39853951470757476)

In [24]:
#cat, inst = '02691156', 'd3b9114df1d8a3388e415c6cf89025f0'
root = '/vision/hwjiang/3d_generation/DeepLocalShapes/examples/all_computed_rot_quad_last/Reconstructions_fix/18/Meshes/ShapeNetV2'
cats = os.listdir(root)

for cat in cats:
    insts = os.listdir(os.path.join(root, cat))
    insts = [it.replace('.ply', '') for it in insts]
    for inst in insts:
        print(cat, inst)
        mesh_gt = load_gt_mesh(get_path(cat, inst, type='gt'))
        center, scale = compute_normalization_parameters(mesh_gt)

        mesh_deepsdf = load_mesh(get_path(cat, inst, type='deepsdf'))
        mesh_deepsdf_norm = normalize_mesh(mesh_deepsdf, scale, center)

        mesh_deepls = load_mesh(get_path(cat, inst, type='deepls'))
        mesh_deepls_norm = normalize_mesh(mesh_deepls, scale, center)

        mesh_recon = load_mesh(get_path(cat, inst, type='ours'))
        mesh_recon_norm = normalize_mesh(mesh_recon, scale, center)

        save_path = './examples/save/shapenet/{}/{}'.format(cat, inst)
        os.makedirs(save_path, exist_ok=True)

        with open(os.path.join(save_path, 'gt.obj'), 'w') as f:
            mesh_gt.export(f, file_type='obj')

        with open(os.path.join(save_path, 'deepsdf.obj'), 'w') as f:
            mesh_deepsdf_norm.export(f, file_type='obj')

        with open(os.path.join(save_path, 'deepls.obj'), 'w') as f:
            mesh_deepls_norm.export(f, file_type='obj')

        with open(os.path.join(save_path, 'recon.obj'), 'w') as f:
            mesh_recon_norm.export(f, file_type='obj')


03001627 cc1b4eb1a9164e04f06885bd08de3f64
03001627 cbe006da89cca7ffd6bab114dd47e3f
03001627 ce055b7a4cccc7e182db9fca4b68095
03001627 ce387113cca9c0754d33a7fd9c37a7d2
03001627 cc8fe2000b1471b2a85f7c85e000fc79
03001627 cc665438c4f7baab137d9700e13a503f
03001627 cbcc5cd1aec9f3413aa677469bbdd68c
03001627 cd7674dec40cb30f30e06a011e63236a
03211117 6856b84c3a8463e3d5f7c62c7dfe63f9
03211117 4abc3db68398e2cf16eb1b6d0ba9133c
03211117 6c8f7736660f2e97e441f5bb6f88ca61
02924116 f413b641dfdbf6317f8c317a43ebc941
02933112 e3f3d58fe3cd9746a485daf2a997e28
02933112 6d5049bc983aa8b4a805114916841d69
02933112 48b9fbf28fd33df5c59350d819542ec7
03467517 73368b2f443e51661082f2ea630bf69e
03467517 1c8c6874c0cb9bc73429c1c21d77499d
03467517 1758c5cbffc7f6634236d3c4918b91c6
03636649 d34a10201a5448a253cf897b7fc1d12
03636649 d190d56a1e6213e420ebca9c2a65e97d
03636649 d3f84442201b6c137fb7f79251dd7727
03636649 d83fc71f3978130e335fe03ac3704320
03636649 daaca6107846a666a8737fcf23389014
03636649 d2d76338428129f62ffd6bbc7610a

In [20]:
def fast_load_obj(file_obj, **kwargs):
    """
    Code slightly adapted from trimesh (https://github.com/mikedh/trimesh)
    Thanks to Michael Dawson-Haggerty for this great library !
    loads an ascii wavefront obj file_obj into kwargs
    for the trimesh constructor.
    vertices with the same position but different normals or uvs
    are split into multiple vertices.
    colors are discarded.
    parameters
    ----------
    file_obj : file object
                   containing a wavefront file
    returns
    ----------
    loaded : dict
                kwargs for trimesh constructor
    """
    # make sure text is utf-8 with only \n newlines
    text = file_obj.read()
    if hasattr(text, 'decode'):
        text = text.decode('utf-8')
    text = text.replace('\r\n', '\n').replace('\r', '\n') + ' \n'
    meshes = []
    def append_mesh():
        # append kwargs for a trimesh constructor
        # to our list of meshes
        if len(current['f']) > 0:
            # get vertices as clean numpy array
            vertices = np.array(
                current['v'], dtype=np.float64).reshape((-1, 3))
            # do the same for faces
            faces = np.array(current['f'], dtype=np.int64).reshape((-1, 3))
            # get keys and values of remap as numpy arrays
            # we are going to try to preserve the order as
            # much as possible by sorting by remap key
            keys, values = (np.array(list(remap.keys())),
                            np.array(list(remap.values())))
            # new order of vertices
            vert_order = values[keys.argsort()]
            # we need to mask to preserve index relationship
            # between faces and vertices
            face_order = np.zeros(len(vertices), dtype=np.int64)
            face_order[vert_order] = np.arange(len(vertices), dtype=np.int64)
            # apply the ordering and put into kwarg dict
            loaded = {
                'vertices': vertices[vert_order],
                'faces': face_order[faces],
                'metadata': {}
            }
            # build face groups information
            # faces didn't move around so we don't have to reindex
            if len(current['g']) > 0:
                face_groups = np.zeros(len(current['f']) // 3, dtype=np.int64)
                for idx, start_f in current['g']:
                    face_groups[start_f:] = idx
                loaded['metadata']['face_groups'] = face_groups
            # we're done, append the loaded mesh kwarg dict
            meshes.append(loaded)
    attribs = {k: [] for k in ['v']}
    current = {k: [] for k in ['v', 'f', 'g']}
    # remap vertex indexes {str key: int index}
    remap = {}
    next_idx = 0
    group_idx = 0
    for line in text.split("\n"):
        line_split = line.strip().split()
        if len(line_split) < 2:
            continue
        if line_split[0] in attribs:
            # v, vt, or vn
            # vertex, vertex texture, or vertex normal
            # only parse 3 values, ignore colors
            attribs[line_split[0]].append([float(x) for x in line_split[1:4]])
        elif line_split[0] == 'f':
            # a face
            ft = line_split[1:]
            if len(ft) == 4:
                # hasty triangulation of quad
                ft = [ft[0], ft[1], ft[2], ft[2], ft[3], ft[0]]
            for f in ft:
                # loop through each vertex reference of a face
                # we are reshaping later into (n,3)
                if f not in remap:
                    remap[f] = next_idx
                    next_idx += 1
                    # faces are "vertex index"/"vertex texture"/"vertex normal"
                    # you are allowed to leave a value blank, which .split
                    # will handle by nicely maintaining the index
                    f_split = f.split('/')
                    current['v'].append(attribs['v'][int(f_split[0]) - 1])
                current['f'].append(remap[f])
        elif line_split[0] == 'o':
            # defining a new object
            append_mesh()
            # reset current to empty lists
            current = {k: [] for k in current.keys()}
            remap = {}
            next_idx = 0
            group_idx = 0
        elif line_split[0] == 'g':
            # defining a new group
            group_idx += 1
            current['g'].append((group_idx, len(current['f']) // 3))
    if next_idx > 0:
        append_mesh()
    return meshes